### 1. data set preprocessing

* 원본 데이터는 다양한 class가 뒤죽박죽 섞여있었음. 이를, class별로 정리하는 전처리
* fold0 = air_conditioner
* fold1 = car_horn
* fold2 = children_playing
* fold3 = dog_bark
* fold4 = drilling
* fold5 = engine_idling
* fold6 = gun_shot
* fold7 = jackhammer
* fold8 = siren
* fold9 = street_music

In [ ]:
import glob
import shutil

arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name = ["air_conditioner", "car_horn", "children_playing", "dog_bark",
       "drilling", "engine_idlling", "gun_shot", "jackhammer", "siren", "street_music"]

def name(num):
    arr[num] = arr[num] + 1
    return Name[num] + str(arr[num])

In [ ]:
for i in range(1, 11):
    folderName = "fold"+str(i)
    fileList = glob.glob("1_origindata/"+folderName+"/*.wav")
    for j in range(len(fileList)):
        tempSplit = fileList[j].split("-")
        originfileName = fileList[j]
        newfileName = "2_data/fold"+str(tempSplit[1])+"/"+name(int(tempSplit[1]))+".wav"
        shutil.copy(originfileName, newfileName)
        print("Success copy", end = " : ")
        print(originfileName + " -> " + newfileName)
    print("/// " + folderName + " finish..")
print("all finish...")

### 2. convert wav to jpg
* librosa 라이브러리르 이용하여 wav파일을 jpg파일로 변환

* 시각화 결과 0, 2, 5, 7, 9번 class는 무의미하고 판별이 어려울 것으로 판단
* 1, 3, 4, 6, 8번 class로 classifier 개발 시작. + 위의 5개 class는 배경 백색 소음 역할 수행

* 즉, 디렉토리는 0 to 9가 아닌, 0 to 4로 재 정렬된다.
* fold0 : car_horn
* fold1 : dog_bark
* fold2 : drilling
* fold3 : gun_shot
* fold4 : siren

* 단, 학습 완료 후, 프로젝트 진행이 안정화되었을 때, 10개 class판별을 위한 upgrade 진행을 할 수 있으며, 이를 위한 origin data는 보관한다.

In [ ]:
import librosa
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import glob
%matplotlib inline

In [ ]:
for i in range(10):
    folderName = "fold" + str(i)
    print("======"+folderName+"======")
    fileList = glob.glob("2_data/"+folderName+"/*.wav")
    for j in range(len(fileList)):
        tempSplit = fileList[j].split(".")
        tempSplit2 = tempSplit[0].split("\\")
        fileName = tempSplit2[-1]
        
        plt.figure()
        y, sr = librosa.load(fileList[j])
        S = np.abs(librosa.stft(y))
        librosa.display.specshow(librosa.power_to_db(S**2, ref = np.max),
                                 sr = sr, y_axis = 'log', x_axis = 'time')
        #plt.colorbar(format = '%+2.0f dB')
        plt.title('Log-Power spectrogram')
        #plt.show()
        print("Save file Complete : 3_image/"+folderName+"/"+fileName+".jpg")
        plt.savefig("3_image/"+folderName+"/"+fileName+".jpg")
        plt.close()

In [ ]:
# 시각화 origin code backup
"""
plt.figure()
y, sr = librosa.load(fileList[0])
S = np.abs(librosa.stft(y))
librosa.display.specshow(librosa.power_to_db(S**2, ref = np.max),
                         sr = sr, y_axis = 'log', x_axis = 'time')
plt.colorbar(format = '%+2.0f dB')
plt.title('Log-Power spectrogram')
plt.show()
"""

### 3. jpg 사진 자르기
* 필요한 부분만 자르기
* 여기서 0, 2, 5, 7, 9는 배경 class로 선정, 즉 fold5에 모아서 저장한다
* 단, 모든 데이터를 저장하면 5000장으로 다른 데이터셋에 비해 과하게 많은 양을 차지하므로 1/5로 축소하여 사용한다

In [ ]:
from PIL import Image
from tqdm import tqdm_notebook
import glob
"""
img = Image.open("test.jpg")
area = (54, 32, 385, 253)
cropped_img = img.crop(area)
cropped_img.save("crop.jpg")
"""

In [ ]:
for i in tqdm_notebook(range(6)):
    folderName = "fold" + str(i)
    print("======"+folderName+"======")
    fileList = glob.glob("3_image/"+folderName+"/*.jpg")
    for j in range(len(fileList)):
        fileName = fileList[j].split("\\")[1]
        img = Image.open(fileList[j])
        area = (54, 32, 385, 253)
        cropped_img = img.crop(area)
        cropped_img.save("4_imageData/"+folderName+"/"+fileName)
        print("Image Split complete : " + fileName)

### split dataset
* 통합 database를 train, validation, test로 분할
* 전체 데이터 10이라 할때, train(6), val(2), test(2)

In [ ]:
import glob
import shutil

for i in range(6):
    folderName = "fold" + str(i)
    fileList = glob.glob("4_imageData/"+folderName+"/*.jpg")
    trainRange = int(len(fileList)*0.6)
    validationRange = int(len(fileList)*0.2)
    
    print("train : 0:"+str(trainRange))
    print("validation : "+str(trainRange+1)+":"+str(trainRange+validationRange))
    print("test : "+str(trainRange+validationRange+1)+":"+str(len(fileList)))
    
    # train data
    for j in range(trainRange+1):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]
        
        newfileName = "5_dataset/train/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)

    # validation data
    for j in range(trainRange+1, trainRange + validationRange + 1):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]        
        
        newfileName = "5_dataset/val/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)
        
    # test data
    for j in range(trainRange + validationRange + 1, len(fileList)):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]        
        
        newfileName = "5_dataset/test/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)        

In [ ]:
import glob

print("class 별 data set 개수 측정")
print("========== car_horn ==========")
print('train car_horn image count : ' + str(len(glob.glob("5_dataset/train/fold0/*jpg"))))
print('validation car_horn image count : ' + str(len(glob.glob("5_dataset/val/fold0/*jpg"))))
print('test car_horn image count : ' + str(len(glob.glob("5_dataset/test/fold0/*jpg"))))

print("========== dog_bark ==========")
print('train dog_bark image count : ' + str(len(glob.glob("5_dataset/train/fold1/*jpg"))))
print('validation dog_bark image count : ' + str(len(glob.glob("5_dataset/val/fold1/*jpg"))))
print('test dog_bark image count : ' + str(len(glob.glob("5_dataset/test/fold1/*jpg"))))

print("========== drilling ==========")
print('train drilling image count : ' + str(len(glob.glob("5_dataset/train/fold2/*jpg"))))
print('validation drilling image count : ' + str(len(glob.glob("5_dataset/val/fold2/*jpg"))))
print('test drilling image count : ' + str(len(glob.glob("5_dataset/test/fold2/*jpg"))))

print("========== gun_shot ==========")
print('train gun_shot image count : ' + str(len(glob.glob("5_dataset/train/fold3/*jpg"))))
print('validation gun_shot image count : ' + str(len(glob.glob("5_dataset/val/fold3/*jpg"))))
print('test gun_shot image count : ' + str(len(glob.glob("5_dataset/test/fold3/*jpg"))))

print("========== siren ==========")
print('train siren image count : ' + str(len(glob.glob("5_dataset/train/fold4/*jpg"))))
print('validation siren image count : ' + str(len(glob.glob("5_dataset/val/fold4/*jpg"))))
print('test siren image count : ' + str(len(glob.glob("5_dataset/test/fold4/*jpg"))))

print("========== 배경음 ==========")
print('train siren image count : ' + str(len(glob.glob("5_dataset/train/fold5/*jpg"))))
print('validation siren image count : ' + str(len(glob.glob("5_dataset/val/fold5/*jpg"))))
print('test siren image count : ' + str(len(glob.glob("5_dataset/test/fold5/*jpg"))))

## CNN with keras ====================================
### 여기서부터 실행하면 됨

### jpg to numpy

In [3]:
import glob
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

class_num = 6

train_files = glob.glob("5_dataset/train/*/*.jpg")
val_files = glob.glob("5_dataset/val/*/*.jpg")
test_files = glob.glob("5_dataset/test/*/*.jpg")

y_temp_train = []
y_temp_validation = []
y_temp_test = []

train_temp_files = []
val_temp_files = []
test_temp_files = []

for _file in train_files:
    train_temp_files.append(_file)
    _label = _file.split("\\")
    y_temp_train.append(int(_label[1][4]))

for _file in val_files:
    val_temp_files.append(_file)
    _label = _file.split("\\")
    y_temp_validation.append(int(_label[1][4]))    

for _file in test_files:
    test_temp_files.append(_file)
    _label = _file.split("\\")
    y_temp_test.append(int(_label[1][4]))  
    
image_width = 331
image_height = 221
channels = 3

x_train = np.ndarray(shape = (len(train_temp_files), image_width, image_height, channels), dtype = np.float32)
y_train = np.ndarray(shape = (len(y_temp_train), class_num), dtype = np.float32)
x_validation = np.ndarray(shape = (len(val_temp_files), image_width, image_height, channels), dtype = np.float32)
y_validation = np.ndarray(shape = (len(y_temp_validation), class_num), dtype = np.float32)
x_test = np.ndarray(shape = (len(test_temp_files), image_width, image_height, channels), dtype = np.float32)
y_test = np.ndarray(shape = (len(y_temp_test), class_num), dtype = np.float32)

In [4]:
def one_hot_encoding(num):
    arr = [0, 0, 0, 0, 0, 0]
    for i in range(class_num):
        if (i==num):
            arr[i] = 1
        else:
            arr[i] = 0
    return arr

In [5]:
i = 0
for _file in train_files:
    img = load_img(_file)
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((331, 221, 3))
    x = (x-128.0)/128.0
    x_train[i] = x
    y_train[i] = one_hot_encoding(y_temp_train[i])
    i += 1
print("Train data converting complete")

i = 0
for _file in val_files:
    img = load_img(_file)
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((331, 221, 3))
    x = (x-128.0)/128.0
    x_validation[i] = x
    y_validation[i] = one_hot_encoding(y_temp_validation[i])
    i += 1
print("validation data converting complete")

i = 0
for _file in test_files:
    img = load_img(_file)
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((331, 221, 3))
    x = (x-128.0)/128.0
    x_test[i] = x
    y_test[i] = one_hot_encoding(y_temp_test[i])
    i += 1
print("test data converting complete")

Train data converting complete
validation data converting complete
test data converting complete


In [6]:
print("x_train : " + str(type(x_train)))
print("y_train : " + str(type(y_train)))
print("x_validation : " + str(type(x_validation)))
print("y_validation : " + str(type(y_validation)))
print("x_test : " + str(type(x_test)))
print("y_test : " + str(type(y_test)))

x_train : <class 'numpy.ndarray'>
y_train : <class 'numpy.ndarray'>
x_validation : <class 'numpy.ndarray'>
y_validation : <class 'numpy.ndarray'>
x_test : <class 'numpy.ndarray'>
y_test : <class 'numpy.ndarray'>


### CNN with keras
* ref url
* https://datascienceschool.net/view-notebook/51e147088d474fe1bf32e394394eaea7/
* http://pythonstudy.xyz/python/article/402-numpy-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0
* https://snowdeer.github.io/machine-learning/2018/01/10/convolution-neural-network/

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
import keras

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l1, l2

import matplotlib.pyplot as plt
import os
import numpy

### model1 : CRM CRM CRM CRM FDD, epoch = 15, batch_size = 100
### model2 : CRM CRM CRM CRM FDD, epoch = 15, batch_size = 30
### model3 : CRM CRM CRM CRM FDD, epoch = 15, batch_size = 50 - good
### model4 : CRM CRM CRM CRM Drop(0.25) FDD, epoch = 15, batch_size = 50
### model5 : CRM CRM Drop(0.25) CRM CRM  FDD, epoch = 15, batch_size = 50
### model6 : CRM CRM + L2reg(0.001) CRM CRM FDD, epoch = 15, batch_size = 50
### model7 : CRM CRM + L2reg(0.001) CRM CRM Dropout(0.5) FDD, epoch = 15, batch_size = 50
### model8 : 

In [ ]:
model_num = 8

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(331, 221, 3))) # input size : 331, 221, 3
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(64, (3, 3), activity_regularizer = l2(0.001), activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(class_num, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy',
             optimizer = optimizers.RMSprop(lr = 1e-4),
             metrics = ['acc'])

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=15, batch_size=50, 
                    validation_data=(x_validation, y_validation),
                    verbose=1)

In [ ]:
model.save('5classSoundModel'+str(model_num)+'.h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation loss')
plt.legend()
plt.savefig("5classSoundModelGraph"+str(model_num)+"-1.png")

plt.figure()

plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.savefig("5classSoundModelGraph"+str(model_num)+"-2.png")

plt.show()

### 실제 test set에 대해 잘못된 판별 갯수 파악

In [ ]:
import glob
from tqdm import tqdm_notebook
testList = glob.glob("5_dataset/test/*/*.jpg")
total = [0, 0, 0, 0, 0, 0]
result = [0, 0, 0, 0, 0, 0]
print("Length : " + str(len(testList)))
for i in tqdm_notebook(range(len(testList))):
    predictNum = model.predict_classes(x_test[i:i+1, :], verbose = 0)
    _ArrRealNum = testList[i].split("\\")
    realNum = int(_ArrRealNum[1].split("fold")[-1])
    total[realNum] = total[realNum] + 1
    if(predictNum != realNum):
        result[realNum] = result[realNum] + 1
print("total : " , end = "")
print(total)
print("result : " , end = "")
print(result)
print("error% : ", end = "")
for i in range(6):
    temp = result[i] / total[i]
    temp = temp * 100
    print(round(temp, 3), end = ", ")

In [18]:
num = [0, 0, 0, 0, 0, 0]
for i in y_test:
    for j in range(6):
        if(int(i[j]) == 1):
            num[j] += 1
for i in range(1, 6):
    num[i] = num[i] + num[i-1]
print("0 to " + str(num[0]))
print(str(num[0]+1) + " to " + str(num[1]))
print(str(num[1]+1) + " to " + str(num[2]))
print(str(num[2]+1) + " to " + str(num[3]))
print(str(num[3]+1) + " to " + str(num[4]))
print(str(num[4]+1) + " to " + str(num[5]))

0 to 86
87 to 285
286 to 484
485 to 559
560 to 745
746 to 944


In [ ]:
num = 0
for i in range(len(testList)):
    print(str(num), end = " : ")
    print(model.predict_classes(x_test[i:i+1, :], verbose = 0), end = " , ")
    if(i%10==0 and i != 0):
        print("\n")
    if(i==86 or i == 285 or i == 484 or i == 559 or i == 745):
        print("\n===========================================================================")
        print("\n")
    num += 1

### Keras ensemble

In [17]:
import glob
from keras.models import load_model
from tqdm import tqdm_notebook
fileList = glob.glob("6_ModelBackUp/*.h5")
testList = glob.glob("5_dataset/test/*/*.jpg")

models = list()
for file in tqdm_notebook(fileList):
    models.append(load_model(file))
    
num = 0
for i in tqdm_notebook(range(len(testList))):
    classPredict = [0, 0, 0, 0, 0, 0]
    answer = 5
    maximum = 0
    maxIndex = 0
    for model in models:
        tempPredict = model.predict_classes(x_test[i:i+1, :], verbose = 0)
        classPredict[int(tempPredict)] += 1
    for j in range(6):
        if(classPredict[j] > maximum):
            overlap = 0
            maximum = classPredict[j]
            maxIndex = j
    print(num, end = " -> ")
    num += 1
    print(classPredict, end = " : ")
    answer = maxIndex
    print(answer)
    if(i==86 or i == 285 or i == 484 or i == 559 or i == 745):
        print("\n===========================================================================")
        print("\n")

0 -> [7, 0, 0, 0, 0, 0] : 0
1 -> [7, 0, 0, 0, 0, 0] : 0
2 -> [7, 0, 0, 0, 0, 0] : 0
3 -> [7, 0, 0, 0, 0, 0] : 0
4 -> [7, 0, 0, 0, 0, 0] : 0
5 -> [7, 0, 0, 0, 0, 0] : 0
6 -> [7, 0, 0, 0, 0, 0] : 0
7 -> [6, 0, 0, 0, 0, 1] : 0
8 -> [7, 0, 0, 0, 0, 0] : 0
9 -> [5, 0, 0, 0, 0, 2] : 0
10 -> [4, 0, 0, 0, 3, 0] : 0
11 -> [7, 0, 0, 0, 0, 0] : 0
12 -> [7, 0, 0, 0, 0, 0] : 0
13 -> [7, 0, 0, 0, 0, 0] : 0
14 -> [7, 0, 0, 0, 0, 0] : 0
15 -> [7, 0, 0, 0, 0, 0] : 0
16 -> [1, 0, 0, 0, 2, 4] : 5
17 -> [1, 0, 0, 0, 1, 5] : 5
18 -> [7, 0, 0, 0, 0, 0] : 0
19 -> [7, 0, 0, 0, 0, 0] : 0
20 -> [7, 0, 0, 0, 0, 0] : 0
21 -> [7, 0, 0, 0, 0, 0] : 0
22 -> [0, 0, 0, 0, 0, 7] : 5
23 -> [0, 2, 0, 0, 0, 5] : 5
24 -> [7, 0, 0, 0, 0, 0] : 0
25 -> [7, 0, 0, 0, 0, 0] : 0
26 -> [5, 0, 2, 0, 0, 0] : 0
27 -> [6, 0, 1, 0, 0, 0] : 0
28 -> [1, 0, 4, 0, 0, 2] : 2
29 -> [0, 0, 0, 0, 0, 7] : 5
30 -> [3, 0, 2, 0, 0, 2] : 0
31 -> [1, 0, 0, 0, 1, 5] : 5
32 -> [2, 0, 2, 0, 0, 3] : 5
33 -> [2, 0, 2, 0, 0, 3] : 5
34 -> [7, 0, 0, 0, 0, 0]

274 -> [0, 6, 0, 0, 0, 1] : 1
275 -> [0, 7, 0, 0, 0, 0] : 1
276 -> [0, 7, 0, 0, 0, 0] : 1
277 -> [0, 7, 0, 0, 0, 0] : 1
278 -> [0, 0, 6, 0, 1, 0] : 2
279 -> [0, 7, 0, 0, 0, 0] : 1
280 -> [0, 7, 0, 0, 0, 0] : 1
281 -> [0, 7, 0, 0, 0, 0] : 1
282 -> [0, 7, 0, 0, 0, 0] : 1
283 -> [0, 7, 0, 0, 0, 0] : 1
284 -> [0, 7, 0, 0, 0, 0] : 1
285 -> [0, 0, 7, 0, 0, 0] : 2



286 -> [0, 0, 7, 0, 0, 0] : 2
287 -> [0, 0, 7, 0, 0, 0] : 2
288 -> [0, 0, 7, 0, 0, 0] : 2
289 -> [0, 0, 7, 0, 0, 0] : 2
290 -> [0, 0, 7, 0, 0, 0] : 2
291 -> [0, 0, 7, 0, 0, 0] : 2
292 -> [0, 0, 7, 0, 0, 0] : 2
293 -> [0, 1, 6, 0, 0, 0] : 2
294 -> [0, 0, 7, 0, 0, 0] : 2
295 -> [0, 0, 7, 0, 0, 0] : 2
296 -> [0, 0, 7, 0, 0, 0] : 2
297 -> [0, 0, 7, 0, 0, 0] : 2
298 -> [0, 0, 5, 2, 0, 0] : 2
299 -> [0, 0, 2, 0, 2, 3] : 5
300 -> [0, 0, 7, 0, 0, 0] : 2
301 -> [0, 0, 7, 0, 0, 0] : 2
302 -> [0, 0, 7, 0, 0, 0] : 2
303 -> [0, 0, 7, 0, 0, 0] : 2
304 -> [0, 0, 7, 0, 0, 0] : 2
305 -> [0, 0, 6, 0, 0, 1] : 2
306 -> [0, 0, 7, 0, 0, 0] : 2
307 -> 

541 -> [0, 0, 0, 7, 0, 0] : 3
542 -> [0, 0, 0, 6, 0, 1] : 3
543 -> [0, 0, 0, 5, 0, 2] : 3
544 -> [0, 0, 0, 7, 0, 0] : 3
545 -> [0, 0, 0, 6, 0, 1] : 3
546 -> [0, 0, 1, 6, 0, 0] : 3
547 -> [0, 1, 1, 5, 0, 0] : 3
548 -> [0, 0, 0, 7, 0, 0] : 3
549 -> [0, 0, 0, 7, 0, 0] : 3
550 -> [0, 0, 0, 7, 0, 0] : 3
551 -> [0, 0, 0, 7, 0, 0] : 3
552 -> [0, 0, 0, 7, 0, 0] : 3
553 -> [0, 0, 0, 7, 0, 0] : 3
554 -> [0, 1, 0, 6, 0, 0] : 3
555 -> [0, 0, 0, 7, 0, 0] : 3
556 -> [0, 0, 0, 7, 0, 0] : 3
557 -> [0, 0, 0, 7, 0, 0] : 3
558 -> [0, 0, 0, 7, 0, 0] : 3
559 -> [0, 0, 0, 0, 7, 0] : 4



560 -> [0, 0, 0, 0, 7, 0] : 4
561 -> [0, 0, 0, 0, 7, 0] : 4
562 -> [0, 0, 0, 0, 7, 0] : 4
563 -> [0, 0, 0, 0, 7, 0] : 4
564 -> [0, 0, 0, 0, 7, 0] : 4
565 -> [0, 0, 0, 0, 5, 2] : 4
566 -> [0, 0, 0, 0, 7, 0] : 4
567 -> [0, 0, 0, 0, 7, 0] : 4
568 -> [0, 0, 0, 0, 7, 0] : 4
569 -> [0, 0, 0, 0, 7, 0] : 4
570 -> [0, 0, 0, 0, 7, 0] : 4
571 -> [0, 0, 0, 0, 7, 0] : 4
572 -> [0, 0, 0, 0, 7, 0] : 4
573 -> [0, 0, 0, 0, 7, 0] : 4
574 -> 

808 -> [0, 4, 3, 0, 0, 0] : 1
809 -> [0, 4, 3, 0, 0, 0] : 1
810 -> [1, 4, 2, 0, 0, 0] : 1
811 -> [2, 2, 3, 0, 0, 0] : 2
812 -> [2, 0, 0, 0, 2, 3] : 5
813 -> [0, 1, 1, 0, 0, 5] : 5
814 -> [0, 0, 0, 0, 0, 7] : 5
815 -> [0, 0, 0, 0, 0, 7] : 5
816 -> [0, 0, 1, 2, 0, 4] : 5
817 -> [2, 4, 1, 0, 0, 0] : 1
818 -> [0, 0, 2, 0, 0, 5] : 5
819 -> [0, 0, 0, 0, 0, 7] : 5
820 -> [0, 0, 0, 0, 0, 7] : 5
821 -> [0, 0, 0, 0, 0, 7] : 5
822 -> [0, 0, 0, 0, 0, 7] : 5
823 -> [0, 0, 0, 0, 0, 7] : 5
824 -> [1, 0, 0, 0, 0, 6] : 5
825 -> [0, 0, 0, 0, 0, 7] : 5
826 -> [0, 0, 0, 0, 1, 6] : 5
827 -> [0, 1, 0, 0, 1, 5] : 5
828 -> [4, 0, 2, 0, 0, 1] : 0
829 -> [0, 0, 0, 0, 1, 6] : 5
830 -> [0, 0, 0, 0, 1, 6] : 5
831 -> [0, 0, 0, 0, 1, 6] : 5
832 -> [0, 0, 0, 0, 1, 6] : 5
833 -> [1, 0, 4, 0, 0, 2] : 2
834 -> [7, 0, 0, 0, 0, 0] : 0
835 -> [5, 0, 0, 0, 0, 2] : 0
836 -> [7, 0, 0, 0, 0, 0] : 0
837 -> [0, 0, 0, 0, 0, 7] : 5
838 -> [4, 2, 0, 0, 0, 1] : 0
839 -> [6, 0, 1, 0, 0, 0] : 0
840 -> [0, 0, 3, 0, 1, 3] : 5
841 -> [1,

In [24]:
print("test 100개당 잘못된 검출 횟수")
print(round((12/86)*100, 3))
print(round((55/(285-86+1))*100, 3))
print(round((40/(484-285+1))*100, 3))
print(round((4/(559-484+1))*100, 3))
print(round((50/(745-558)) * 100, 3))
print(round((90/(944-745))*100, 3))

test 100개당 잘못된 검출 횟수
13.953
27.5
20.0
5.263
26.738
45.226
